In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests 

# Get list of all URLs to parse

In [3]:
headers = {
    'authority': 'cdn.cookielaw.org',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://www.medhelp.org/',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'script',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}

In [4]:
URL = 'https://www.medhelp.org/forums/Eye-Care/show/90?page=4'

resp1 = requests.get(URL, headers=headers)

In [5]:
type(resp1.status_code)

int

In [6]:
soup = BeautifulSoup(resp1.content, "html.parser")

In [7]:
div_rel = soup.find('div', id='body_container')

In [13]:
for i in div_rel.find_all('h2', class_ = 'subj_title'):
    print(i.a.attrs['href'])

/posts/Eye-Care/regarding-the-difference-in-my-eye-size/show/1668376
/posts/Eye-Care/flashing-lights-in-my-vision/show/224369
/posts/Eye-Care/How-to-correct-vision-after-cataract-surgery-in-only-one-eye/show/1294816
/posts/Eye-Care/Gas-Bubbles/show/1385815
/posts/Eye-Care/Bilateral-Monocular-Diplopia-Help/show/462330
/posts/Eye-Care/Rebound-Hyperemia/show/491565


In [62]:
def extract_urls_from_medhelp(URL, headers):
    
    response = requests.get(URL, headers=headers)
    if response.status_code != 200:
        return -1
    
    soup = BeautifulSoup(response.content, "html.parser")

    div_rel = soup.find('div', id='body_container')

    all_h2_tags = div_rel.find_all('h2', class_ = 'subj_title')

    post_list = []

    for child in all_h2_tags:
        post_list.append( child.a.attrs['href'])
    
    return post_list




In [63]:
extract_urls_from_medhelp('https://www.medhelp.org/forums/Eye-Care/show/90?page=103', headers=headers)

['/posts/Eye-Care/Starburst-after-yag-cap/show/1900237',
 '/posts/Eye-Care/CRT-Ortho-K/show/1900337',
 '/posts/Eye-Care/after-cataract-intra-vitreal-injection/show/1899577',
 '/posts/Eye-Care/Itchy-white-bump-on-caruncle/show/1883521',
 '/posts/Eye-Care/posterior-polar-and-other-issues/show/1898354',
 '/posts/Eye-Care/Floaters/show/1865636']

# For a particular URL, extract the post and the response

In [22]:
post_url = '/posts/Eye-Care/Starburst-after-yag-cap/show/1900237'

In [23]:
website = 'https://www.medhelp.org'

In [42]:
resp_post = requests.get(website+post_url+'?sort_by=oldest', headers=headers)

In [43]:
resp_post.status_code

200

In [44]:
soup = BeautifulSoup(resp_post.content, "html.parser")

In [45]:
div_rel = soup.find('div', id='body_container')

In [46]:
div_rel.find('h1', class_= 'subj_title').text.strip()

'Starburst after yag cap'

In [47]:
div_rel.find('div', id='subject_msg').text.strip()

'Went back to my eye dr , he said I had a piece\xa0\xa0of vitrerous leftover from the yag cap that was hanging on . It is still connected and moving in front of my line of vision he is doing a second yag cap to fix the problem .\xa0\xa0 Brian'

In [49]:
div_rel.find('div', class_='resp_body').text.strip()

'It is possible that a tag or remnant of the capsule is in the line of sight in the central vitreous.\xa0\xa0If this is bothersome to you, a second YAG laser will probably be able to move the fragment out of your central vision.'

In [53]:
div_rel.find('div', class_= 'resp_info').a.text

'Doreen  Fazio, M.D.'

In [56]:
div_rel.find('div', class_= 'resp_info').a.attrs['href']

'/personal_pages/user/2078052'

In [64]:
def extract_details_medhelp_post(post_url, headers):
    website = 'https://www.medhelp.org'
    URL = website + post_url + '?sort_by=oldest'
    resp_post = requests.get(URL , headers=headers)
    if resp_post.status_code != 200:
        return -1
    soup = BeautifulSoup(resp_post.content, "html.parser")
    div_rel = soup.find('div', id='body_container')

    post_heading = div_rel.find('h1', class_= 'subj_title').text.strip()
    post_details = div_rel.find('div', id='subject_msg').text.strip()

    reply_details = div_rel.find('div', class_='resp_body').text.strip()

    reply_name = div_rel.find('div', class_= 'resp_info').a.text
    reply_page = div_rel.find('div', class_= 'resp_info').a.attrs['href']

    return [post_heading, post_details, reply_details, reply_name, reply_page]

In [65]:
extract_details_medhelp_post(post_url=post_url, headers=headers)

['Starburst after yag cap',
 'Went back to my eye dr , he said I had a piece\xa0\xa0of vitrerous leftover from the yag cap that was hanging on . It is still connected and moving in front of my line of vision he is doing a second yag cap to fix the problem .\xa0\xa0 Brian',
 'It is possible that a tag or remnant of the capsule is in the line of sight in the central vitreous.\xa0\xa0If this is bothersome to you, a second YAG laser will probably be able to move the fragment out of your central vision.',
 'Doreen  Fazio, M.D.',
 '/personal_pages/user/2078052']